In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import pandas as pd

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
sampled_training = pd.read_csv('./drive/MyDrive/VK_hack/final_bert_sampled_train_df.csv')

In [7]:
import numpy as np

In [9]:
clean_train_ready_df = pd.read_csv('./drive/MyDrive/VK_hack/df_extra_gen_old.csv')
# tr_indx = np.load('tr_indx.npy')
# vl_indx = np.load('vl_indx.npy')

In [10]:
clean_train_ready_df.columns

Index(['url', 'title', 'text', 'topic', 'tags', 'date', 'Красота', 'Политика',
       'Экономика', 'Мода', 'Общество', 'Закон и право', 'Кино', 'Телевидение',
       'Персоны', 'События', 'Бренды', 'Наука', 'Гаджеты', 'Соцсети', 'Дом',
       'Технологии', 'Транспорт', 'X', 'Погода', 'Рецепты', 'Опросы',
       'Головоломки', 'labels', 'actual_labels', 'clean_X', 'cut_labels'],
      dtype='object')

In [31]:
clean_train_ready_df.shape

(97241, 32)

In [11]:
train_df = clean_train_ready_df.copy()

In [14]:
extra_train_df = clean_train_ready_df[~clean_train_ready_df.X.isin(sampled_training.X)]

In [15]:
extra_train_df.shape

(2843, 32)

In [20]:
(extra_train_df.X.isna()).sum()

0

In [16]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [17]:
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [18]:
# #Sentences we want sentence embeddings for
# sentences = ['Привет! Как твои дела?',
#              'А правда, что 42 твое любимое число?']

In [21]:
texts_list = list(extra_train_df.loc[:, 'X'].str.replace(u'\xa0', u' ').values)

In [22]:
import numpy as np
from tqdm import tqdm

results = np.zeros(shape=(len(texts_list), 1024))

In [23]:
from torch.utils.data import DataLoader, TensorDataset

device = 'cuda'
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [24]:
for i in texts_list:
    if not isinstance(i, str):
        print(i)

In [25]:
# texts_list_first = texts_list[:len(texts_list) // 2]
# texts_list_second = texts_list[len(texts_list) // 2: ]

In [26]:
tokenized_texts = tokenizer(texts_list, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Create DataLoader
dataset = TensorDataset(tokenized_texts["input_ids"], tokenized_texts["attention_mask"])
dataloader = DataLoader(dataset, batch_size=int(128))

In [27]:
from tqdm import tqdm
import torch

In [28]:
# Generate embeddings
embeddings = []
with torch.no_grad():
    for input_ids, attention_mask in tqdm(dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(outputs, attention_mask)
        embeddings.append(sentence_embeddings.cpu())

# Concatenate embeddings
first_embeddings = torch.cat(embeddings)

100%|██████████| 23/23 [01:23<00:00,  3.63s/it]


In [29]:
torch.save(first_embeddings, './drive/MyDrive/VK_hack/extra_embeddings.pt')

In [30]:
first_embeddings.shape

torch.Size([2843, 1024])

In [ ]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('')

# # Sentences we want to encode. Example:
# sentence = ['This framework generates embeddings for each input sentence']

# # Sentences are encoded by calling model.encode()
# embedding = model.encode(sentence)